### Arboles de regresion
* la diferencia es que el target puede ser una variable continua, numerica; por contra los otros arboles lo que hacen es clasificar
* cuando hay una regresion no lineal muy complicada podemos siempre usar este

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# ejemplo de arboles de decision es el dataset de Boston para predecir precios casas
url = r"https://raw.githubusercontent.com/joanby/python-ml-course/master/datasets/boston/Boston.csv"
data = pd.read_csv(url)
data.head(3)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7


In [5]:
# podemos hacerlo tambien son sklearn train test split
col_names = data.columns.values.tolist()
predictors = col_names[:13]
target = col_names[13] # col names va de 0 a 13 ¡
X = data[predictors]
Y = data[target]


In [7]:
from sklearn.tree import DecisionTreeRegressor

In [8]:
regtree = DecisionTreeRegressor(min_samples_split=10,min_samples_leaf=10,random_state=0)

In [9]:
regtree.fit(X,Y)

DecisionTreeRegressor(min_samples_leaf=10, min_samples_split=10, random_state=0)

In [10]:
preds = regtree.predict(data[predictors])
data["preds"] = preds

In [11]:
# ahora comparamos (vemos que mucha se repiten porque caen en la misma rama del arbol)
data[["preds","medv"]].head(5)

,preds,medv
0,22.840000,24.0
1,22.840000,21.6
2,37.190000,34.7
3,33.753846,33.4
4,33.753846,36.2


In [15]:
# ahora hacemos validacion cruzado para ver si el modelo es consistente
from sklearn.model_selection import KFold
cv = KFold(n_splits=10,shuffle=True,random_state=1)

from sklearn.model_selection import cross_val_score
score = cross_val_score(regtree,X,Y, scoring='neg_mean_squared_error',cv=cv,n_jobs=1) # aqui no usamos accuracy sino mean squared

In [18]:
print(score)
scor = np.mean(score)
print(scor)

[-13.56126026 -15.81815238 -16.80851914 -43.84847746  -9.83021945
 -17.25544008 -14.72465011 -32.41301362 -22.14347973  -9.53776745]
-19.59409796803987


In [ ]:
# esto nos dicen que las previsoines estan por encima o por debajo 19 puntos de las reales

In [22]:
# vemos la importancia de las features (vemos truco imprimir horzional * y sep=\n)
print(*list(zip(predictors,regtree.feature_importances_)),sep="\n")

('crim', 0.033333095852660116)
('zn', 0.013127929346484539)
('indus', 0.0011307722576663543)
('chas', 0.0)
('nox', 0.022601410608427585)
('rm', 0.6159856905321107)
('age', 0.018480937834828907)
('dis', 0.005432740754594164)
('rad', 0.0)
('tax', 0.0023089475509547024)
('ptratio', 0.009677423859204467)
('black', 0.0)
('lstat', 0.27792105140306866)


### Random forest (podemos usarlo tambien para regresion o clasificacion)
* es como un cajon desastre en el mundo de los modelos
* no necesita validacion cruzada ya que utiliza el sistema de bagging
* se toman n observaciones para cada datset(sample) con reemplazo.
* estos datasets se llaman muestras de bootstrap o bolsas
* construimos un arbol de regresion para cada muestras con su correspondinte sample de variable predictora
* la predicion final es el promedio de las observaciones de todos los arboles y para clasificacion, la clase con mayoria de votos en el conjunto de los arboles


In [23]:
# vamos a hacerlo para el dataset de antes (primero una regresion)
from sklearn.ensemble import RandomForestRegressor

In [24]:
# n_jobs = numero de tareas en paralelo que se ejecuta al mismo tiempo
# oob score = true el modelo hace un muestro aleatorio
# n_estimators = numero de arboles que queramos

forest = RandomForestRegressor(n_jobs=2,oob_score=True,n_estimators=10)
forest.fit(X,Y)

RandomForestRegressor(n_estimators=10, n_jobs=2, oob_score=True)

In [25]:
# podemos ver las predicciones 
data["rforest_pred"] = forest.oob_prediction_
data[["rforest_pred","medv"]]

,rforest_pred,medv
0,28.025000,24.0
1,22.825000,21.6
2,34.983333,34.7
3,36.780000,33.4
4,37.300000,36.2
...,...,...
501,28.700000,22.4
502,22.750000,20.6
503,26.800000,23.9
504,26.750000,22.0


In [35]:
# vemos el error cuadratico medio, aqui no hace falta cross validation
sum((data["medv"]-data["rforest_pred"])**2) / len(data)

22.397944043081665

In [38]:
np.mean((data["medv"]-data["rforest_pred"])**2)

22.397944043081665

In [32]:
forest.oob_score_ # similar al coef de determinacion r2

0.7346829921535208

In [40]:
from sklearn.metrics import r2_score
r2_score(Y,forest.oob_prediction_)

0.7346829921535208

In [42]:
## podemos hacer tambien random forest clasification (por ejemplo para iris)
from sklearn import datasets
iris = datasets.load_iris()
X,Y = iris.data, iris.target

In [43]:
from sklearn.preprocessing import MinMaxScaler
max = MinMaxScaler()
X = max.fit_transform(X)

In [44]:
df_iris = pd.DataFrame(X)

In [46]:
from sklearn.ensemble import RandomForestClassifier
rf_clas = RandomForestClassifier(n_jobs=2,oob_score=True,n_estimators=100)
rf_clas.fit(df_iris,Y)

RandomForestClassifier(n_jobs=2, oob_score=True)

In [47]:
rf_clas.oob_score_  

0.9466666666666667

### consideracions
* suelen funcionar bastante bien
* no hay que podar los arboeles, por lo que podemos obtener buenso resultados con bosques grandes
* parametros:
    * tamaño del nodo(min_samples_leaf) podemos jugar con este parametro (de serie = 1)
    * numero arboels (n_estimators) aqui podemso de media 500 (no hace falta podarlos)

Al momento de ajustar el modelo, debemos tener en cuenta los siguientes hiperparámetros. Estos nos ayudarán a que el bosque de mejores resultados para cada ejercicio. Recuerda que esto no se trata de “copiar y pegar”!

* n_estimators: será la cantidad de árboles que generaremos.
* max_features: la manera de seleccionar la cantidad máxima de features para cada árbol.
* min_sample_leaf: número mínimo de elementos en las hojas para permitir un nuevo split (división) del nodo.
* oob_score: es un método que emula el cross-validation en árboles y permite mejorar la precisión y evitar overfitting.
* boostrap: para utilizar diversos tamaños de muestras para entrenar. Si se pone en falso, * utilizará siempre el dataset completo.
* n_jobs: si tienes multiples cores en tu CPU, puedes indicar cuantos puede usar el modelo al entrenar para acelerar el entrenamiento.

https://mlcourse.ai/articles/topic5-part2-rf/